In [15]:
links = ["https://www.careers360.com/colleges/government-women-polytechnic-college-bharatpur",
         "https://www.careers360.com/colleges/government-women-polytechnic-college-jaipur",
         "https://www.careers360.com/colleges/government-residential-womens-polytechnic-college-jodhpur",
         "https://www.careers360.com/colleges/government-womens-polytechnic-college-kota",
         "https://www.careers360.com/colleges/government-women-polytechnic-college-sanganer",
         ]

In [3]:
len("https://www.careers360.com/colleges/")

36

In [4]:
links[0][36:]

'government-polytechnic-college-ajmer'

In [2]:
import requests
from bs4 import BeautifulSoup
import csv

In [5]:
def clean_section(section):
    # List of unwanted tags to remove (customize as needed)
    unwanted_tags = ["read_more", "single_apply_with_caption"]

    # Loop through each unwanted tag and decompose (remove it)
    for tag_class in unwanted_tags:
        unwanted = section.findAll("div", {"class": tag_class})
        for tag in unwanted:
            tag.decompose()  # Remove the unwanted tag from the section

    # Now return the cleaned-up section content
    return section.get_text(strip=True)

In [9]:
def home_cleaned(target):
  page_code = requests.get(target)
  page_text = page_code.text
  page_soup = BeautifulSoup(page_text, 'html.parser').find ('div',{"class":"college_main_container"})
  Nav_links = page_soup.find("div", {"class": "subNav"}).findAll("li")

  dig_in_links = []
  for li in Nav_links:
      a_tag = li.find("a")
      if a_tag:
          link = a_tag['href']
          link = "https://www.careers360.com" + link
          dig_in_links.append(link)

  section_blk = page_soup.findAll('div', {"class" :"section_blk"})
  sections = []
  for sec in section_blk:
    if sec.find("div", {"class":"read_more"}):
      if sec.find("div", {"class":"read_more"}).find("a"):
        continue;
    sections.append(sec)

  cleaned_sections = []
  for sec in sections:
      cleaned_data = clean_section(sec)
      if cleaned_data:
          cleaned_sections.append(cleaned_data)
  return (target[36:],{"cleaned_home": cleaned_sections, "dig_in": dig_in_links[1:]})

In [16]:
def course_cleaned(target):
  page_code = requests.get(target)
  page_text = page_code.text
  page_soup = BeautifulSoup(page_text, 'html.parser').find ('div',{"class":"course_list"})
  if not page_soup:
    return []
  courses = page_soup.findAll('div',{"class":"card_blk"})
  cleaned_courses = []
  for course in courses:
    title = course.getText()
    link = "https://www.careers360.com/" + course.find('a')['href']
    course_code = requests.get(link)
    course_text = course_code.text
    course_soup = BeautifulSoup(course_text, 'html.parser').findAll('div',{"class":"certificate_block"})
    info = {}
    for block in course_soup:
      heading = block.get("data-heading", "").strip()  # Get the heading of each block

      # For Seats
      if heading == "Quick Facts":
          seats = block.find("div", {"class": "right_btm"})
          if seats:
              info["Seats"] = seats.text.strip()

      # For Course Details
      elif heading == "Course Details":
          details = block.findAll("div", {"class": "course_detail_para"})
          for detail in details:
              mode = detail.find("span", text="Full time")  # Extracting the Mode
              duration = detail.find("span")  # Duration is also inside a span
              if mode:
                  info["Mode"] = mode.text.strip()
              if duration:
                  info["Duration"] = duration.text.strip()

      # For Eligibility Criteria
      elif heading == "Eligibility Criteria":
          eligibility = block.find("div", {"class": "data_html_blk"})
          if eligibility:
              info["Eligibility Criteria"] = eligibility.text.strip()
    cleaned_courses.append({"title": title, "info": info})
  return cleaned_courses

In [17]:
final_data=[]
for link in links:
  data = home_cleaned(link)
  name = data[0]
  Home_data = data[1]["cleaned_home"]
  dig_in_links = data[1]["dig_in"]
  for l in dig_in_links:
    if "courses" in l:
      cleaned_courses = course_cleaned(l)

  final_data.append((name, Home_data, cleaned_courses))

<ipython-input-16-362fe17f383b>:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  mode = detail.find("span", text="Full time")  # Extracting the Mode


In [18]:
final_data

[('government-women-polytechnic-college-bharatpur',
  ['Government Women Polytechnic College, BharatpurHighlightsParameterDescriptionEstablished2010Courses1Degrees and6CoursesInstitute TypeAffiliated CollegeApprovalsAICTEStudent Count63Faculty Count15',
   'ExploreGovernment Women Polytechnic College, BharatpurCoursesDiplomaEngineering and ArchitectureAnimation and DesignBrowse Popular CoursesDiploma in Electrical EngineeringStudy ModeFull timeSeats30Get InfoDiploma in Cyber Forensics and Information SecurityStudy ModeFull timeSeats30Get InfoDiploma in Computer Science and EngineeringStudy ModeFull timeSeats30Get InfoDiploma in Textile DesignStudy ModeFull timeSeats30Get InfoDiploma in Fashion DesigningStudy ModeFull timeSeats30Get InfoDiploma in Costume Design and Dress MakingStudy ModeFull timeSeats30Get InfoView All6CoursesDownload Course List',
   'Government Women Polytechnic College, BharatpurFacilitiesAlumni AssociationsSportsAuditoriumLibraryLaboratoriesMedical/HospitalGymI.T I

In [13]:
data_1 = final_data

In [19]:
data_2 = final_data

In [20]:
data_1

[('government-polytechnic-college-ajmer',
  ['Government Polytechnic College, AjmerHighlightsParameterDescriptionEstablished1958Courses1Degrees and8CoursesInstitute TypeAffiliated CollegeApprovalsAICTEStudent Count797Faculty Count50Campus Size54acres',
   'ExploreGovernment Polytechnic College, AjmerCoursesDiplomaEngineering and ArchitectureBrowse Popular CoursesDiploma in Mechanical EngineeringStudy ModeFull timeSeats40Get InfoDiploma in Electronics EngineeringStudy ModeFull timeSeats40Get InfoDiploma in Electrical EngineeringStudy ModeFull timeSeats40Get InfoDiploma in Civil Engineering Self Finance System ModeStudy ModeFull timeSeats40Get InfoDiploma in Civil EngineeringStudy ModeFull timeSeats30Get InfoDiploma in Printing TechnologyStudy ModeFull timeSeats20Get InfoView All8CoursesDownload Course List',
   'Government Polytechnic College, AjmerFacilitiesGirls HostelLibraryLaboratoriesI.T InfrastructureBoys HostelSportsAuditoriumCafeteriaGuest Room/Waiting RoomView All Facilities',


In [21]:
data_2

[('government-women-polytechnic-college-bharatpur',
  ['Government Women Polytechnic College, BharatpurHighlightsParameterDescriptionEstablished2010Courses1Degrees and6CoursesInstitute TypeAffiliated CollegeApprovalsAICTEStudent Count63Faculty Count15',
   'ExploreGovernment Women Polytechnic College, BharatpurCoursesDiplomaEngineering and ArchitectureAnimation and DesignBrowse Popular CoursesDiploma in Electrical EngineeringStudy ModeFull timeSeats30Get InfoDiploma in Cyber Forensics and Information SecurityStudy ModeFull timeSeats30Get InfoDiploma in Computer Science and EngineeringStudy ModeFull timeSeats30Get InfoDiploma in Textile DesignStudy ModeFull timeSeats30Get InfoDiploma in Fashion DesigningStudy ModeFull timeSeats30Get InfoDiploma in Costume Design and Dress MakingStudy ModeFull timeSeats30Get InfoView All6CoursesDownload Course List',
   'Government Women Polytechnic College, BharatpurFacilitiesAlumni AssociationsSportsAuditoriumLibraryLaboratoriesMedical/HospitalGymI.T I